# This notebook will be used for the Coursera Capstone Course project.

In [1]:
import pandas as pd
import numpy as np

print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Introduction/Business Problem

#### Police and first responders are looking for a model to help predict the severity of car accidents in order to better utilize their resources and more efficiently and effectively deploy personnel to each accident reported. In order to do so, they have hired a data scientist to analyze data compiled in Seattle City to see if there is a way to predict based on weather and road conditions in order to help better deploy resources, warn the public for high severity days, and ultimately save lives.

#### The question they have asked is: can you predict based on weather and road conditions the likely severity of a car accident?



# Data

#### The dataset used for this anaylsis is from the Seattle City Police Department and includes all of the collusions, as recorded by Traffic Records, from 2004 to present.

#### It includes 37 attributes and is labeled by severity of the accidents, describing the fatality of each accident. It includes: a severity, code 0 (unknown) to 3 (fatal), as well as the type of collision, # of people/objects involved, date/time of the accident, junction/lane type where the accident occured, road conditions, influence of the driver (attention, under the influence, speeding), and weather.

#### As weather and road conditions will help the police to send out public warnings or announcements to help caution drivers on days where more severe accidents are likely, the initial focus will be on these attributes.

#### To help further deploy appropriate resoureces, secondary considerations will be on the day/time of accident, junction type/crosswalk involvement, and the number of people/objects involved. If this information can be gathered from the initial accident report within a reasonable amount of time, it could further help the police to deploy appropriate types and amount of assistance to the scene of the accident. 

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_25a75b0fe6734a97b285b33b88264068 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vUtWzKGYLQzFnOe6aTyrHJ4_db7aEKZh2dUJTQPzjb2P',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_25a75b0fe6734a97b285b33b88264068.get_object(Bucket='courseracapstone-donotdelete-pr-awppagh6tidzrr',Key='Collisions.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Seattle_Data = pd.read_csv(body)
# print(Seattle_Data.info)
print(Seattle_Data.head())

            X          Y  OBJECTID  INCKEY  COLDETKEY REPORTNO   STATUS  \
0 -122.356511  47.517361         1  327920     329420  3856094  Matched   
1 -122.361405  47.702064         2   46200      46200  1791736  Matched   
2 -122.317414  47.664028         3    1212       1212  3507861  Matched   
3 -122.318234  47.619927         4  327909     329409  EA03026  Matched   
4 -122.351724  47.560306         5  104900     104900  2671936  Matched   

       ADDRTYPE   INTKEY                                           LOCATION  \
0  Intersection  34911.0                      17TH AVE SW AND SW ROXBURY ST   
1         Block      NaN     HOLMAN RD NW BETWEEN 4TH AVE NW AND 3RD AVE NW   
2         Block      NaN  ROOSEVELT WAY NE BETWEEN NE 47TH ST AND NE 50T...   
3  Intersection  29054.0                           11TH AVE E AND E JOHN ST   
4         Block      NaN  WEST MARGINAL WAY SW BETWEEN SW ALASKA ST AND ...   

   ... ROADCOND                LIGHTCOND PEDROWNOTGRNT SDOTCOLNUM SPEEDING

In [32]:
import numpy as np

#drop rows of data with 'weather', 'road condition', or 'severity code' blank
Seattle_Data.dropna(subset=["WEATHER"], axis=0, inplace=True)
Seattle_Data.dropna(subset=["ROADCOND"], axis=0, inplace=True)
Seattle_Data.dropna(subset=["SEVERITYCODE"], axis=0, inplace=True)

#Set index to ObjectID
Seattle_Data.set_index('OBJECTID')

#Create new dataframe with only the columns required for analysis
SD = Seattle_Data[['OBJECTID', 'WEATHER', 'SEVERITYCODE', 'ROADCOND']]

SD.head()
SD.info

<bound method DataFrame.info of         OBJECTID   WEATHER SEVERITYCODE ROADCOND
0              1     Clear            1      Dry
1              2   Raining            1      Wet
2              3     Clear            2      Dry
3              4   Raining            2      Wet
4              5     Clear            2      Ice
5              6     Clear            1      Dry
6              7  Overcast            1      Dry
7              8     Clear            1      Dry
8              9     Clear           2b      Dry
9             10     Clear            1      Wet
11            12     Clear            1      Dry
12            13     Clear            2      Dry
13            14     Clear            2      Dry
14            15   Unknown            1  Unknown
15            16     Clear            2      Dry
16            17     Clear            1      Dry
17            18   Unknown            1  Unknown
19            20  Overcast            1      Dry
20            21   Raining           

In [4]:
#no longer needed
#Seattle_Data["SEVERITYCODE"] = Seattle_Data["SEVERITYCODE"].astype("object")
#Seattle_Data["OBJECTID"] = Seattle_Data["OBJECTID"].astype("object")
#Seattle_Data["INCKEY"] = Seattle_Data["INCKEY"].astype("object")
#Seattle_Data["COLDETKEY"] = Seattle_Data["COLDETKEY"].astype("object")
#Seattle_Data["INTKEY"] = Seattle_Data["INTKEY"].astype("object")
#Seattle_Data["SEGLANEKEY"] = Seattle_Data["SEGLANEKEY"].astype("object")
#Seattle_Data["CROSSWALKKEY"] = Seattle_Data["CROSSWALKKEY"].astype("object")

In [27]:
SD.WEATHER.unique()

array(['Clear', 'Raining', 'Overcast', 'Unknown', 'Other', 'Snowing',
       'Sleet/Hail/Freezing Rain', 'Fog/Smog/Smoke', 'Blowing Sand/Dirt',
       'Severe Crosswind', 'Partly Cloudy'], dtype=object)

In [39]:
SD.drop(SD.index[SD['WEATHER'] == 'Unknown'], inplace = True)
SD.drop(SD.index[SD['WEATHER'] == 'Other'], inplace = True)


SD.WEATHER.unique()

array(['Clear', 'Raining', 'Overcast', 'Snowing',
       'Sleet/Hail/Freezing Rain', 'Fog/Smog/Smoke', 'Blowing Sand/Dirt',
       'Severe Crosswind', 'Partly Cloudy'], dtype=object)

In [40]:
SD.drop(SD.index[SD['ROADCOND'] == 'Unknown'], inplace = True)
SD.drop(SD.index[SD['ROADCOND'] == 'Other'], inplace = True)


SD.ROADCOND.unique()

array(['Dry', 'Wet', 'Ice', 'Snow/Slush', 'Standing Water',
       'Sand/Mud/Dirt', 'Oil'], dtype=object)

In [54]:
#Assign number values to the Weather and Road Condition columns according to severity of condition
#weather_map = {'Clear': 0, 'Partly Cloudy': 1, 'Overcast': 2, 'Raining': 3, 'Fog/Smog/Smoke': 4, 'Blowing Sand/Dirt': 5, 'Sleet/Hail/Freezing Rain': 6, 'Snowing': 7, 'Severe Crosswind': 8}
#road_map = {'Dry': 0, 'Wet': 1, 'Sand/Mud/Dirt': 2, 'Standing Water': 3, 'Snow/Slush': 4, 'Oil': 5, 'Ice': 6}

#SD.replace({'WEATHER': weather_map, 'ROADCOND': road_map})

,OBJECTID,WEATHER,SEVERITYCODE,ROADCOND
0,1,0,1,0
1,2,3,1,1
2,3,0,2,0
3,4,3,2,1
4,5,0,2,6
5,6,0,1,0
6,7,2,1,0
7,8,0,1,0
8,9,0,2b,0
9,10,0,1,1


In [55]:
SD.head()

#Change column data types
#SD["SEVERITYCODE"] = SD['SEVERITYCODE'].astype(int)
#SD["ROADCOND"] = SD["ROADCOND"].astype(int)
#SD["WEATHER"] = SD["WEATHER"].astype(int)



,OBJECTID,WEATHER,SEVERITYCODE,ROADCOND
0,1,Clear,1,Dry
1,2,Raining,1,Wet
2,3,Clear,2,Dry
3,4,Raining,2,Wet
4,5,Clear,2,Ice


In [45]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

pearson_coef, p_value = stats.pearsonr(SD['WEATHER'], SD['SEVERITYCODE'])
print("The Pearson Correlation Coefficient is", pearson_coef, " with a P-value of P =", p_value)  

TypeError: unsupported operand type(s) for /: 'str' and 'int'